Install the required libraries for the code

In [1]:
!pip install selenium
!pip install pandas
!pip install tqdm
!pip install nltk
!pip install textstat


DEPRECATION: Loading egg at /home/user44/anaconda3/lib/python3.11/site-packages/face_recognition-1.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/user44/anaconda3/lib/python3.11/site-packages/face_recognition-1.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/user44/anaconda3/lib/python3.11/site-packages/face_recognition-1.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /home/user44/anaconda3/lib/python3.11/site-packages/face_

Import the libraries

In [2]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.webdriver.common.by import By
import re
import time
from tqdm import tqdm
import textstat
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import os

In [3]:
nltk.download('punkt')  #run only one time to download the necessary libraries.

[nltk_data] Downloading package punkt to /home/user44/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Web Scraping


In [4]:
def extraction(title, paragraph):
    try:
        df = pd.read_excel('./dataset/Input.xlsx', sheet_name='Sheet1')
    except Exception as e:
        print("Error reading Excel file:", e)
        return
    
    driver = webdriver.Chrome()
    
    for i in tqdm(range(len(df['URL']))):
        url = df['URL'][i]
        
        try:
            driver.get(url)
        except Exception as e:
            print(f"Error accessing URL '{url}':", e)
            continue
        
        try:
            title_element = driver.find_element(By.TAG_NAME, "h1")
            title = title_element.text.strip()
        except Exception as e:
            print("Error getting title for URL:", url, e)
            title = ""
        
        try:
            para_elements = driver.find_elements(By.CLASS_NAME, "tagdiv-type")
            if para_elements:
                para = para_elements[0].text.strip()
                sentence_list = para.split(".")[:-1]
                para = ".".join(sentence_list)
            else:
                para = ""
        except Exception as e:
            print("Error getting paragraph for URL:", url, e)
            para = ""
        
        final_sentence = title + "\n" + para
        
        try:
            with open(f"./text_file/{df['URL_ID'][i]}.txt", "w", encoding='utf-8') as f:
                f.write(final_sentence)
        except Exception as e:
            print("Error writing to file for URL_ID:", df['URL_ID'][i], e)
    
    driver.quit()

In [5]:
title="entry-title"
paragraph="tagdiv-type"

sentence=extraction(title,paragraph)

 36%|███████████████                           | 36/100 [01:17<03:17,  3.09s/it]

Error getting title for URL: https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future/ Message: no such element: Unable to locate element: {"method":"tag name","selector":"h1"}
  (Session info: chrome=126.0.6478.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5b0ff8ebb69a <unknown>
#1 0x5b0ff8b9e0dc <unknown>
#2 0x5b0ff8bea931 <unknown>
#3 0x5b0ff8beaa21 <unknown>
#4 0x5b0ff8c2f234 <unknown>
#5 0x5b0ff8c0d89d <unknown>
#6 0x5b0ff8c2c5c3 <unknown>
#7 0x5b0ff8c0d613 <unknown>
#8 0x5b0ff8bdd4f7 <unknown>
#9 0x5b0ff8bdde4e <unknown>
#10 0x5b0ff8e8186b <unknown>
#11 0x5b0ff8e85911 <unknown>
#12 0x5b0ff8e6d35e <unknown>
#13 0x5b0ff8e86472 <unknown>
#14 0x5b0ff8e51cbf <unknown>
#15 0x5b0ff8eab098 <unknown>
#16 0x5b0ff8eab270 <unknown>
#17 0x5b0ff8eba7cc <unknown>
#18 0x710898894ac3 <unknown>



 49%|████████████████████▌                     | 49/100 [01:34<01:29,  1.75s/it]

Error getting title for URL: https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future/ Message: no such element: Unable to locate element: {"method":"tag name","selector":"h1"}
  (Session info: chrome=126.0.6478.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
#0 0x5b0ff8ebb69a <unknown>
#1 0x5b0ff8b9e0dc <unknown>
#2 0x5b0ff8bea931 <unknown>
#3 0x5b0ff8beaa21 <unknown>
#4 0x5b0ff8c2f234 <unknown>
#5 0x5b0ff8c0d89d <unknown>
#6 0x5b0ff8c2c5c3 <unknown>
#7 0x5b0ff8c0d613 <unknown>
#8 0x5b0ff8bdd4f7 <unknown>
#9 0x5b0ff8bdde4e <unknown>
#10 0x5b0ff8e8186b <unknown>
#11 0x5b0ff8e85911 <unknown>
#12 0x5b0ff8e6d35e <unknown>
#13 0x5b0ff8e86472 <unknown>
#14 0x5b0ff8e51cbf <unknown>
#15 0x5b0ff8eab098 <unknown>
#16 0x5b0ff8eab270 <unknown>
#17 0x5b0ff8eba7cc <unknown>
#18 0x710898894ac3 <unknown>



100%|█████████████████████████████████████████| 100/100 [02:40<00:00,  1.61s/it]


The error above is due to the link is expired and the percent 36 and 49 shows the index of the expired link in the excel sheet.

Sentiment analysis

In [6]:
def read_words(file_path, encoding='utf-8'):
    with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
        words = file.read().splitlines()
    return set(words)

def read_stopwords(folder_path, encoding='utf-8'):
    stopwords = set()
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding=encoding, errors='ignore') as file:
                stopwords.update(file.read().splitlines())
    return stopwords

def tokenize_and_filter(sentence, stopwords):
    words = word_tokenize(sentence.lower())
    return [word for word in words if word.isalnum() and word not in stopwords]

def calculate_scores(sentence, positive_words, negative_words, stopwords):
    
    words = tokenize_and_filter(sentence, stopwords)
    
    positive_score = sum(1 for word in words if word in positive_words)
    
    negative_score = sum(1 for word in words if word in negative_words)
    
    total_words = len(words)
    
    epsilon = 1e-6  
    
    polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + epsilon)
    
    subjectivity_score = (positive_score + negative_score) / (total_words + epsilon)
    
    syllables_per_word = textstat.syllable_count(' '.join(words)) / (total_words + epsilon)
    
    complex_words = sum(1 for word in words if textstat.syllable_count(word) >= 3)
    
    percentage_complex_words = (complex_words / (total_words + epsilon)) * 100
    
    average_sentence_length = total_words  
    
    fog_index = 0.4 * (average_sentence_length + percentage_complex_words)
    
    average_word_length = sum(len(word) for word in words) / (total_words + epsilon)
    
    personal_pronouns = sum(1 for word in words if word in ["i", "we", "me", "us", "my", "our", "mine", "ours"])
    
    return {
        'POSITIVE SCORE': positive_score,
        'NEGATIVE SCORE': negative_score,
        'POLARITY SCORE': polarity_score,
        'SUBJECTIVITY SCORE': subjectivity_score,
        'AVG SENTENCE LENGTH': average_sentence_length,
        'PERCENTAGE OF COMPLEX WORDS': percentage_complex_words,
        'FOG INDEX': fog_index,
        'AVG NUMBER OF WORDS PER SENTENCE': total_words,
        'COMPLEX WORD COUNT': complex_words,
        'WORD COUNT': total_words,
        'SYLLABLE PER WORD': syllables_per_word,
        'PERSONAL PRONOUNS': personal_pronouns,
        'AVG WORD LENGTH': average_word_length
    }

# Function to process text files and store results in a DataFrame
def process_text_files(folder_path, positive_words, negative_words, stopwords):
    data = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path):
            with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
                file_content = file.read()
                scores = calculate_scores(file_content, positive_words, negative_words, stopwords)
                if scores:
                    #scores['URL_ID'] = filename
                    data.append(scores)
    return pd.DataFrame(data)

def load(df):
    excel_df = pd.read_excel('./dataset/Input.xlsx', sheet_name='Sheet1')
    final_df = pd.concat([excel_df, df], axis=1)
    final_df.to_csv('Output Data Structure.csv', index=False)
    

In [7]:
# Example sentence# Load positive and negative words
positive_words = read_words('./dataset/MasterDictionary/positive-words.txt', encoding='latin-1')
negative_words = read_words('./dataset/MasterDictionary/negative-words.txt', encoding='latin-1')

# Load stopwords from a folder
stopwords_folder = './dataset/StopWords/'
stopwords = read_stopwords(stopwords_folder, encoding='latin-1')

text_files_folder = './text_file/'

# Process text files and store results in a DataFrame
df = process_text_files(text_files_folder, positive_words, negative_words, stopwords)
load(df)


Checking the result is in required format or not

In [8]:
output_df=pd.read_csv('./Output Data Structure.csv')
output_df.head(25)

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,blackassign0001,https://insights.blackcoffer.com/rising-it-cit...,24,27,-0.058824,0.106250,480,33.333333,205.333333,480,160,480,2.193750,0,7.079167
1,blackassign0002,https://insights.blackcoffer.com/rising-it-cit...,67,41,0.240741,0.111455,969,30.134159,399.653664,969,292,969,2.130031,0,6.793602
2,blackassign0003,https://insights.blackcoffer.com/internet-dema...,68,34,0.333333,0.102616,994,33.601610,411.040644,994,334,994,2.238431,0,7.338028
3,blackassign0004,https://insights.blackcoffer.com/rise-of-cyber...,27,48,-0.280000,0.121556,617,22.042139,255.616856,617,136,617,1.865478,0,6.359806
4,blackassign0005,https://insights.blackcoffer.com/ott-platform-...,28,70,-0.428571,0.120838,811,33.908755,337.963502,811,275,811,2.163995,0,7.246609
5,blackassign0006,https://insights.blackcoffer.com/the-rise-of-t...,37,34,0.042254,0.101429,700,37.285714,294.914286,700,261,700,2.237143,0,7.242857
6,blackassign0007,https://insights.blackcoffer.com/rise-of-cyber...,27,12,0.384615,0.114370,341,27.859237,147.543695,341,95,341,2.096774,0,6.868035
7,blackassign0008,https://insights.blackcoffer.com/rise-of-inter...,32,10,0.523810,0.080925,519,37.957611,222.783044,519,197,519,2.317919,0,7.581888
8,blackassign0009,https://insights.blackcoffer.com/rise-of-cyber...,31,26,0.087719,0.080395,709,34.132581,297.253032,709,242,709,2.249647,0,7.029619
9,blackassign0010,https://insights.blackcoffer.com/rise-of-cyber...,55,16,0.549296,0.086691,819,28.693529,339.077411,819,235,819,2.032967,0,6.892552
